# Utility Notebook A - Seafloor Age and Spreading Rate Grids

This is the first utility notebook. Please ensure [`gplately`](https://github.com/GPlates/gplately.git) is installed on your system using one of the following terminal command methods once you have created a conda environment:

* **conda forge**: 
    ```python 
        conda install conda-forge::gplately
    ```
* **pip**: 
    ```python
       pip install gplately
    ```
    
* **git clone**
    ```python
        conda activate insert_conda_environment_here
        git clone https://github.com/GPlates/gplately.git
        cd gplately
        pip install .
    ```

In [1]:
# Import gplately and pygplates once it is installed
import gplately
from gplately import pygplates

# 1. Upload plate model files

You will need:

1. A rotation model (`.rot` file)
2. Topology features (`.gpml` or `.shp` file(s)) containing all reconstructible plate topologies. (Note: if using `.shp` files ensure that its corresponding `.dbf, .prj, .xml and .shx` files are in the same directory)
3. Continents - (polygons in `.gpml` or `.shp` file(s))

Optional: 
* Static polygons (`.gpml` or `.shp` file(s))
* Coastlines (`.gpml` or `.shp` file(s))


### If your plate model is hosted on GPlately's `DataServer` (see full list [here](https://gplates.github.io/gplately/#dataserver))...
...get the model name string identifier and run the cell below. Otherwise skip this cell.

In [2]:
# Type in the model identifier here
model_id = "Muller2022"

# Use GPlately's DataServer to get a rotation model, topology features and static polygons
gdownload = gplately.download.DataServer(model_id, verbose=True)
rotation_model, topology_features, static_polygons = gdownload.get_plate_reconstruction_files()

# Use GPlately's DataServer to get coastlines and continents
coastlines, continents, COBs = gdownload.get_topology_geometries()

Checking whether the requested files need to be updated...
Requested files are up-to-date!
Checking whether the requested files need to be updated...
Requested files are up-to-date!


# OR 
### If your plate model is NOT hosted on GPlately's `DataServer`...
...provide all paths to the required files in the cell below and run this cell instead.

In [3]:
input_directory = "/Users/laurenilano/Downloads/Cao1.8Ga"

# Change the code to point to the rotation filenames
rotation_filenames = [
    
    input_directory+'/optimisation/1000_0_rotfile_Merdith_et_al_Cao_new_ncc_git_20231016_run1.rot',
    input_directory+'/optimisation/Rotation_file_1800_1000_no_ncc_new_SaoFrancisco_git_20231016_run1.rot'
]

rotation_model = pygplates.RotationModel(rotation_filenames)


# Change the code to point to the topology feature filenames
topology_filenames = [
    input_directory+'/250-0_plate_boundaries_Merdith_et_al.gpml',
    input_directory+'/1000-410-Topologies_Merdith_et_al_Cao.gpml',
    input_directory+'/410-250_plate_boundaries_Merdith_et_al.gpml',
    input_directory+'/1800-1000Ma-plate-boundary_new_valid_time_and_subduction_polarity.gpml',
    input_directory+'/1000-410-Transforms_Merdith_et_al_Cao.gpml',
    input_directory+'/TopologyBuildingBlocks_Merdith_et_al.gpml',
    input_directory+'/1000-410-Convergence_Merdith_et_al_Cao.gpml',
    input_directory+'/1000-410-Divergence_Merdith_et_al_Cao.gpml',
    input_directory+'/Proterozoic_Poles.gpml'
]

# KEEP THESE 3 LINES AS-IS
topology_features = pygplates.FeatureCollection()
for topology_filename in topology_filenames:
    topology_features.add( pygplates.FeatureCollection(topology_filename) )
        

# Change the code to point to the continent filename(s)
continents = input_directory+'/shapes_continents_Cao.gpmlz'

 Once one of the cells above have been run, run this cell, keeping `time` as 1000Ma for now.

In [4]:
model = gplately.PlateReconstruction(rotation_model, topology_features)

time = 1000
gplot = gplately.PlotTopologies(model, time=time, continents=continents)

# 2. Call `SeafloorGrid` to build seafloor points through time

`SeafloorGrid` is a gplately module that reconstructs mid-ocean ridges iteratively through time, and builds new ocean floor points from these ridges. At every timestep, new points emerge, and older points age/move outwards from their origin ridges. Eventually some points reach subduction zones or continent boundaries (from the topology files and the continent file you provided) and they are deleted. The iteration starts at `max_time` and ends at `min_time`.

In [5]:
# The first and last time (in Myr) to build grids
min_time = 990
max_time = 1000

# The interval between successive times (in Myr)
ridge_time_step = 1. 

Other input parameters

In [6]:
# In degrees, the spacing between points tessellated along a ridge. A smaller value is a finer ridge.
ridge_sampling = 0.5

# The degree spacing for the final regular grids
grid_spacing = 0.1

Full path to directory to save grids to - if it doesn't exist, it will be made automatically. Also provide a small string to name the grids, ideally the model's first author's surname and year of production, e.g. `Muller2022`

In [7]:
# This is a folder where all gridding files are saved 
save_directory = "/Volumes/Carbon/OTHER_FILES/Muller2022_v1.2_Grids_Feb3"

# A string to help name files according to a plate model.
file_collection = "Muller2022"

`SeafloorGrid` uses [`stripy`](https://github.com/underworldcode/stripy) which builds the first mesh of ocean points to start the gridding iteration. It requires an integer whose magnitude is proportional to the fineness of the global mesh. We typically use 6.

The first timestep of the gridding iteration is populated with a uniform spreading rate. This creates a uniform profile of seafloor age at the first timestep. Over time, as points emerge from ridges, the plate model builds a proper spreading rate profile because plate velocities are encoded in the model. Most of these initial uniform points erode into trenches or continents through time. The initial rate is typically 100mm/yr. 

In [8]:
# An integer whose magnitude is proportional to the fineness of the global mesh
refinement_levels = 6

# An initial spreading rate to kickstart the recursive gridding process. 
initial_ocean_mean_spreading_rate = 100. #mm/yr

Once we run the workflow and the process is interrupted before the `Reconstruct by topologies: working on time 1000.00 Ma` step, we can recover progress if `resume_from_checkpoints = True`.

Otherwise, if it is interrupted during the stage `Reconstruct by topologies: working on time 1000.00 Ma` or after, we will need to restart the entire notebook as the ocean points are built iteratively.

In [9]:
resume_from_checkpoints = True

#### Continent contouring
If set to `True`, the workflow builds a buffer around continent polygons through time. This will take an additional couple of hours. The default is True.

In [10]:
contour_continents = True

In [11]:
seafloorgrid = gplately.oceanscontouring.SeafloorGrid(
    model, 
    gplot, 
    
    max_time = max_time,
    min_time = min_time,

    ridge_time_step = ridge_time_step,
    ridge_sampling = ridge_sampling,
    grid_spacing = grid_spacing,
    
    save_directory = save_directory,
    file_collection = file_collection,

    refinement_levels = refinement_levels,
    initial_ocean_mean_spreading_rate = initial_ocean_mean_spreading_rate,
    
    resume_from_checkpoints = resume_from_checkpoints,
    contour_continents=contour_continents,
    
    # So far, this does not need to be changed
    zval_names = ['SPREADING_RATE'],
)

# 3. Begin reconstruction by topologies

In [16]:
seafloorgrid.reconstruct_by_topologies()

Preparing all initial files...
Finished building initial_ocean_seed_points!
Creating contoured continent masks (this will take a couple of hours...)
Finished continent contouring: 998.0Ma...
Finished continent contouring: 993.0Ma...
Finished continent contouring: 995.0Ma...
Finished continent contouring: 996.0Ma...
Finished continent contouring: 994.0Ma...
Finished continent contouring: 997.0Ma...
Finished continent contouring: 999.0Ma...
Finished continent contouring: 1000.0Ma...
Finished continent contouring: 992.0Ma...
Finished continent contouring: 990.0Ma...
Finished continent contouring: 991.0Ma...
Saved 1000.0Ma contoured continental masks to netCDF!
Saved 999.0Ma contoured continental masks to netCDF!
Saved 998.0Ma contoured continental masks to netCDF!
Saved 997.0Ma contoured continental masks to netCDF!
Saved 996.0Ma contoured continental masks to netCDF!
Saved 995.0Ma contoured continental masks to netCDF!
Saved 994.0Ma contoured continental masks to netCDF!
Saved 993.0Ma co

In [13]:
seafloorgrid.save_directory

'/Volumes/Carbon/OTHER_FILES/Muller2022_v1.2_Grids_Feb3'

In [14]:
import os
os.path.isdir(save_directory)

True

In [15]:
save_directory

'/Volumes/Carbon/OTHER_FILES/Muller2022_v1.2_Grids_Feb3'